<div align="center">
    <img src="images/logo_kbtu.png" alt="logo_kbtu" />
</div>

### RNN demo

<br />
<br />
November 5, 2022

### Data loading

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r', encoding='utf-8').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1821625 characters, 146 unique.


In [2]:
# I don’t quite understand why in literature lessons schoolchildren are offered to read Dostoevsky’s Crime and Punishment, 
# and not this novel..
print(data[:590])

Annotation

Самый сложный, самый многоуровневый и неоднозначный из романов Достоевского, который критики считали то «интеллектуальным детективом», то «ранним постмодернизмом», то – «лучшим из произведений о загадочной русской душе». Роман, легший в основу десятков экранизаций – от предельно точных до самых отвлеченных, – но не утративший своей духовной силы…



* * *

Посвящается Анне Григорьевне Достоевской


Истинно, истинно говорю вам: если пшеничное зерно, падши в землю, не умрет, то останется одно; а если умрет, то принесет много плода.

(Евангелие от Иоанна, гл. XII, ст. 24).




In [3]:
' '.join(chars)

'ê ’ К é m X Ч ” Я р Г F и 0 P т M с e Ц ( G 9 ь г Э x q 2 g S N n A f î „ — У p Щ ] , ю o i в z ? « Б d Ф 1 – I - Q п ё П к à R х л э Ш В з й : Т » V Ж u c ъ 8 у )   s h l Д 6 D И k м t а ч è Ç М ж ! ; T C Р Х Н y … [ O J “ r b А щ 7 v С н a ‑ Л О ц Е З \n ф е L ш j 5 . Ю U д я B 4 о * 3 ы б'

### Initialization

In [4]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

<div align="left">
    <img src="images/rnn_char_level_example.jpg" width=400/>
</div>

### Loss function

 * forward pass for loss calculation
 * backpropagation of error

Let us denote the components of the probability vector of symbols (classes) at the RNN output as
$$ p_k = \frac{e^{f_k}}{\sum\limits_j e^{f_j}}$$

Then the value of the loss function on the next object $x_i$
$$ L_i = -\ln (p_{y_i})$$

Then, by direct calculation of the derivative, we can obtain
$$ \frac{\partial L_i}{\partial f_k} = p_k - 1(y_i = k)$$

In [5]:
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [6]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

### The main loop

In [7]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [8]:
while n < 10**4 + 1000: #True 
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
  
    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))
  
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 1000 == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                  [dWxh, dWhh, dWhy, dbh, dby], 
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
  
    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 …к8МиR4ЗаOо]«:îcнёG“Оm—sСЧQcD*бфdоîр5z7О!к»Б]ЛvФj1BÇAPЖпёбЦz7Е]ЯП.ИшупàСЧBу.ggzс’Б]3ПВ…ЗЯюЦiApRь-—2РмQeьЧФФо»Oà2вk1Лз,cV7èуyОЮл»xОиНЭèКNЩЕэжV Ш-‑kи;Я PSвu.e1Q ,чыS6пЛКзрG—МbЭИbДeяxш-ёщux8шфрБуbyи.Pця] 
----
iter 0, loss: 124.590160
----
 кенемясЯл Она теробыэтлороу, поком писсиври э сякнед ч в викозавожьшотнасжлетаде в, дот » ченедтадигбыЯере, в всвысибидсзклремниен, и. квим пейвьмеяскя кибоне нтете босвФу  ночстсстоки сндвредоло сяш. 
----
iter 1000, loss: 93.683423
----
 то, м. Сх, с– о по эь,о, Подех рым, м……сжозевей а накОксьме ох См? рнен нык ом как, х ом  зне нажери накакемям, прино омустьже го ны?кова
ь Нопе Семнатьтед ТавоС. С лемнак ннень о..ь, Иох,. мана юютат 
----
iter 2000, loss: 76.426299
----
 еай и – на, по, ’валак, сог вожобы башы Но праа вер ч.. в? о ио, сатам ротоде дара ледавидоде др солтыша ипбо бысосретенушдидгсуло, аеро бdвив аголог; ны вибони себкарагредабисорег ра госе бnдогорарас 
----
iter 3000, loss: 69.911899
----
  никда тоженьет. пренсаз, кажя, Дото

### Till $10^4$ iterations

----

 шîрhUНч-rGexêШнAфРЮuГB“аёЛУz[нОp,tцgйеБ2Ç0q(ЕКжКQêGnlИмxqЩGЕЦ[:qы ЮkщiжxдЭ‑IgGёuД2f–щсйТашèиЕfIФбàXЭтЯРЧКkХЧVjтзЦгyЮчp…1шБ”QgХîÇydиb*”иЧ’,TX.,рuынBUmксхêШ(–XВяGАЭ,M…азNЮ„;SсДбВ
ШДЦ)сg*1n4OСмlБTхT»»Лhè 

----

#### iter 0, loss: 124.590170


----

  это бым вый, из о четоть вына скицса что. Ие стамыу, свложекьмисти сти на риз алум сщитикая темвы, чаль и, к к вычыо ивсе мазая, бывак, у мавеет вымю сол, ельсты с вышенум свотеже былилю казелеу выта 

----


#### iter 5000, loss: 62.775185

----

Подарал прел гикруИ нексЦа оторуга и тоязарнийам нчегага: пуда, погоь, и равка ниголю и семи, отвы Имарачеля, и балая —стотвя не до жегдровитя и ста даяхоль! ят гдескнее  неюл

– Веенить идлю пробыль 

----

#### iter 10000, loss: 59.253950



### Till $10^5$ iterations

----

  продут – обаищести, дебы».

– Тытьщиныма катеры было вашила сви тутя? Возотерия, чута? Повзал его, и впрому с выродился, что голо редел, лоб, верю. В леть еще поребяка, и слод, хонул в деть, вупушул, 

----

**iter 100000, loss: 49.823363**

----

### $5*10^5$ iterations

----

крадание зазаженгий тою отвелось фастие, идо, кождо, что чем пришкоже, я камо кара сих-с теже есем патин иро-нать помне не начанно полили», боло. Если, это-то и дравствы» и молод не дурнать не прогору 

----

**iter 500000, loss: 46.207874**
 